In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'bayanpy', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer'}


In [3]:
book_number = 2
interactions_by_chapter = pickle.load(
    open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "rb")
)

In [4]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [5]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [6]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [7]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,6.386117e-01,0.192563,0.800000,0.822344,0.233101,1.999976e-01,1.999976e-01,76,4925
Ronald Weasley,5.880481e-01,0.098880,0.610526,0.697347,0.131538,1.841676e-01,1.841676e-01,58,2926
Hermione Granger,3.818956e-01,0.044927,0.484211,0.631655,0.069670,1.196025e-01,1.196025e-01,46,1556
Gilderoy Lockhart,1.654270e-01,0.152500,0.473684,0.627111,0.041717,5.180883e-02,5.180883e-02,45,744
Tom Riddle,8.538583e-02,0.042211,0.231579,0.534776,0.017188,2.674157e-02,2.674157e-02,22,368
...,...,...,...,...,...,...,...,...,...
Z. Nettles,3.095933e-99,0.000000,0.010526,0.010526,0.010417,8.291507e-22,1.385491e-18,1,1
Aurora Sinistra,1.977281e-06,0.000000,0.010526,0.327701,0.001714,6.192520e-07,6.192520e-07,1,1
Bloody Baron,3.424333e-04,0.000000,0.010526,0.444737,0.001603,1.072455e-04,1.072455e-04,1,1


In [8]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [9]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [10]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [11]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [12]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [13]:
metrics_df.to_csv(f"data/processed/{book_number}/metrics.csv")

In [13]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [14]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [17]:
nx.write_gexf(G, f"data/processed/{book_number}/graph.gexf")